In [41]:
import numpy as np
import xarray as xr
import glob
import cftime

## Go get extra workers

In [2]:
#for use on Casper
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(cores=36,
                     processes=36, memory="380GB",
                     project='P93300041',
                     walltime='2:00:00')
cluster.scale(36)
client = Client(cluster)

/glade/u/apps/dav/opt/python/3.7.5/gnu/8.3.0/pkg-library/20200417/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [2]:
#for use on cheyenne
from dask_jobqueue import PBSCluster
from dask.distributed import Client
cluster = PBSCluster(cores=32,
                     processes=32, memory="108GB",
                     project='P93300041',
                     queue='regular',
                     resource_spec='select=1:ncpus=36:mem=109G',
                     walltime='02:00:00')
cluster.scale(32)
client = Client(cluster)

In [6]:
client

Client Scheduler: tcp://10.148.10.17:37307 Dashboard: http://10.148.10.17:8787/status,Cluster Workers: 32 Cores: 32 Memory: 108.16 GB


## Define file paths and preprocess function

In [7]:
thedir = '/glade/scratch/charlie/archive/fates_clm50_global_4x5_historicaltransient_2e3f469f_2905a9ba/lnd/hist/'
paths = sorted(glob.glob(thedir+'*.h0.*'))

In [54]:
data_vars =['SECONDARY_FOREST_FRACTION',
            'PATCH_AREA_BY_AGE',
            'PFTcanopycrownarea',
            'SECONDARY_AREA_PATCH_AGE_DIST',
            'SECONDARY_AREA_AGE_ANTHRO_DIST',
            'NPLANT_SCLS',
            'ED_biomass',
            'area',
            'WOOD_PRODUCT',
            'GPP',
            'NPP']
def preprocess(ds):
    '''specify a smaller set of data_vars'''
    return ds[data_vars]

## Read in the data

In [55]:
%%time 
ds=xr.open_mfdataset(paths,combine='by_coords',decode_cf=False,
                     preprocess=preprocess,parallel=True)
time_vector = cftime.num2date(ds['time'].values-16,units=ds['time'].attrs['units'],
                calendar=ds['time'].attrs['calendar'])
ds['time']=xr.DataArray(time_vector,attrs=ds['time'].attrs)

CPU times: user 27.3 s, sys: 724 ms, total: 28.1 s
Wall time: 45.5 s


## If you want to add some more variables later

In [56]:
data_vars =['TWS']
def preprocess(ds):
    '''specify a smaller set of data_vars'''
    return ds[data_vars]

In [57]:
%%time 
ds2=xr.open_mfdataset(paths,combine='by_coords',decode_cf=False,
                      preprocess=preprocess,parallel=True)
time_vector = cftime.num2date(ds2['time'].values-16,units=ds2['time'].attrs['units'],
                calendar=ds2['time'].attrs['calendar'])
ds2['time']=xr.DataArray(time_vector,attrs=ds2['time'].attrs)
for v in data_vars:
    ds[v]=ds2[v]

CPU times: user 16.3 s, sys: 544 ms, total: 16.8 s
Wall time: 31.9 s


In [58]:
ds

,Array,Chunk
Bytes,26.39 MB,13.25 kB
Shape,"(1992, 46, 72)","(1, 46, 72)"
Count,5976 Tasks,1992 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,184.73 MB,92.74 kB
Shape,"(1992, 7, 46, 72)","(1, 7, 46, 72)"
Count,5976 Tasks,1992 Chunks
Type,float32,numpy.ndarray


In [63]:
##make sure you get equal number of months
for mm in 1+np.arange(12):
    print((ds['time.month']==mm).sum().values)

166
166
166
166
166
166
166
166
166
166
166
166


In [66]:
ds['time.month'][0]

<xarray.DataArray 'month' ()>
array(1)
Coordinates:
    time     object 1850-01-16 00:00:00